**Advanced Astroinformatics (Semester 1 2024)**
# Discussing Results so far

**Advanced Astroinformatics Student Project**

*N. Hernitschek*



---
## Contents
* [Recap, Questions](#first-bullet)
* [Tasks for Today](#second-bullet)
* [Summary](#fifth-bullet)


## 1. Recap, Questions <a class="anchor" id="first-bullet"></a>

Time for questions!

Your **tasks until this week** were:


Create a feature table showing the calculated features for all variable stars in the TESS sample (using the _TESS_lightcurves_outliercleaned light curves).

Use this feature table to create a triangle plot showing the features first for all variable stars from one star type (e.g.: ACV). If this works, for each type create a triangle plot.
Then, calculate the feature table also for the other versions of the light curves (_TESS_lightcurves_median_after_detrended, _TESS_lightcurves_raw). Overplot these features to each of your triangle plots. (I.e.: After this step you should have individual triangle plots for the types ACV, CEP, DCEP..., showing the calculated features for te raw, median/detrended, and outlier cleaned light curves.)

What do these triangle plots tell you?

Did you find certain patterns, regions... in the plots that correspond to what you have read about variable stars (https://www.aavso.org/vsx/index.php?view=about.vartypes)?

If already done: Which distinctive differences do you recognize between plots done from the three different versions of the light curve (_TESS_lightcurves_outliercleaned, _TESS_lightcurves_median_after_detrended, _TESS_lightcurves_raw)?

## 2. Tasks for Today <a class="anchor" id="second-bullet"></a>

If not already done: Repeat the light curve plots, feature table and triangle plots for all the other versions of the light curves (_TESS_lightcurves_median_after_detrended, _TESS_lightcurves_raw).

Otherwise: Take a look at https://scikit-learn.org/stable/tutorial/basic/tutorial.html and try to familiarize yourself with `scikit-learn`.

You can also take a look at https://www.overleaf.com/. This is an online LaTeX editor. It is widely used in astronomy for writing papers collaboratively. I thus recommend using it for your project report.

Today we are doing (again) a coding and Q&A session today and continue next week with a detailed introduction to machine learning and `scikit-learn`.

In [3]:
import os
import pandas as pd
import feets
import corner
import matplotlib.pyplot as plt

# Define the base data directory
base_data_dir = "/home/user/DAA11"

# Define the data directories for different versions of light curves
data_dirs = {
    'outliercleaned': os.path.join(base_data_dir, "TESS_lightcurves_outliercleaned"),
    'median_detrended': os.path.join(base_data_dir, "TESS_lightcurves_median_after_detrended"),
    'raw': os.path.join(base_data_dir, "TESS_lightcurves_raw")
}

# Define the star types
star_types = ["ACV", "CEP", "DCEP"]  # Add more star types as needed

# Define the features to exclude
exclude_features = ["AndersonDarling", "StetsonK", "StetsonK_AC"]

# Initialize the feature space
fs = feets.FeatureSpace(data=['time', 'magnitude', 'error'], exclude=exclude_features)

# Function to calculate features for a light curve file
def calculate_features(file_path):
    try:
        lc_data = pd.read_csv(file_path, delim_whitespace=True, header=None, comment='#')
        lc_data.columns = ['time', 'magnitude', 'error']
        
        lc_data['time'] = pd.to_numeric(lc_data['time'], errors='coerce')
        lc_data['magnitude'] = pd.to_numeric(lc_data['magnitude'], errors='coerce')
        lc_data['error'] = pd.to_numeric(lc_data['error'], errors='coerce')
        
        lc_data.dropna(inplace=True)
        
        features, values = fs.extract(time=lc_data['time'].values,
                                      magnitude=lc_data['magnitude'].values,
                                      error=lc_data['error'].values)
        
        return pd.Series(values, index=features)
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.Series()

# Dictionary to store feature tables for each star type and each data version
feature_tables = {data_version: {star_type: pd.DataFrame() for star_type in star_types} for data_version in data_dirs}

# Load light curves and calculate features for each data version
for data_version, data_dir in data_dirs.items():
    for star_type in star_types:
        star_type_dir = os.path.join(data_dir, star_type)
        feature_list = []
        
        # Check if the directory exists before processing
        if os.path.exists(star_type_dir):
            # Process a limited number of files
            num_files_to_process = 5
            
            lc_files = [lc_file for lc_file in os.listdir(star_type_dir) if lc_file.endswith(".lc")][:num_files_to_process]
            
            for lc_file in lc_files:
                lc_file_path = os.path.join(star_type_dir, lc_file)
                features = calculate_features(lc_file_path)
                if not features.empty:
                    feature_list.append(features)
        
        feature_tables[data_version][star_type] = pd.DataFrame(feature_list)

# Output feature tables to CSV files (optional)
for data_version, table in feature_tables.items():
    for star_type, df in table.items():
        output_csv = f"feature_table_{data_version}_{star_type}.csv"
        df.to_csv(output_csv, index=False)

# Function to create and save triangle plots
def create_triangle_plot(feature_data, labels, output_file, title):
    figure = corner.corner(feature_data, labels=labels, show_titles=True, title_kwargs={"fontsize": 12})
    figure.suptitle(title)
    figure.savefig(output_file)
    plt.close()

# Create triangle plots for each star type and each data version
for data_version, table in feature_tables.items():
    for star_type, df in table.items():
        feature_data = df.values  # Get the values from DataFrame
        if feature_data.shape[0] <= feature_data.shape[1]:
            print(f"Not enough samples for {data_version} - {star_type}. Skipping plot generation.")
            continue
        
        output_file = f"triangle_plot_{data_version}_{star_type}.png"
        title = f"Feature Triangle Plot for {star_type} Stars ({data_version})"
        
        try:
            create_triangle_plot(feature_data, fs.features, output_file, title)
        except AssertionError as e:
            print(f"AssertionError occurred for {data_version} - {star_type}: {e}")

print("Processing completed.")


Error processing /home/user/DAA11/TESS_lightcurves_outliercleaned/ACV/308452159_sector01_4_3_cleaned.lc: Length mismatch: Expected axis has 1 elements, new values have 3 elements
Error processing /home/user/DAA11/TESS_lightcurves_outliercleaned/ACV/270304671_sector01_1_3_cleaned.lc: Length mismatch: Expected axis has 1 elements, new values have 3 elements
Error processing /home/user/DAA11/TESS_lightcurves_outliercleaned/ACV/92705248_sector01_1_1_cleaned.lc: Length mismatch: Expected axis has 1 elements, new values have 3 elements
Error processing /home/user/DAA11/TESS_lightcurves_outliercleaned/ACV/364424408_sector01_4_2_cleaned.lc: Length mismatch: Expected axis has 1 elements, new values have 3 elements
Error processing /home/user/DAA11/TESS_lightcurves_outliercleaned/ACV/238869272_sector01_3_2_cleaned.lc: Length mismatch: Expected axis has 1 elements, new values have 3 elements
Error processing /home/user/DAA11/TESS_lightcurves_outliercleaned/CEP/278863994_sector01_4_4_cleaned.lc: L

/home/user/anaconda3/lib/python3.7/site-packages/feets/extractors/ext_eta_e.py:115: RuntimeWarning: divide by zero encountered in true_divide
  w = 1.0 / np.power(np.subtract(time[1:], time[:-1]), 2)
/home/user/anaconda3/lib/python3.7/site-packages/feets/extractors/ext_eta_e.py:125: RuntimeWarning: invalid value encountered in double_scalars
  time[0], 2) * S1 / (sigma2 * S2 * N ** 2))
/home/user/anaconda3/lib/python3.7/site-packages/feets/extractors/ext_max_slope.py:91: RuntimeWarning: divide by zero encountered in true_divide
  slope = np.abs(magnitude[1:] - magnitude[:-1]) / (time[1:] - time[:-1])
/home/user/anaconda3/lib/python3.7/site-packages/feets/extractors/ext_eta_e.py:115: RuntimeWarning: divide by zero encountered in true_divide
  w = 1.0 / np.power(np.subtract(time[1:], time[:-1]), 2)
/home/user/anaconda3/lib/python3.7/site-packages/feets/extractors/ext_eta_e.py:125: RuntimeWarning: invalid value encountered in double_scalars
  time[0], 2) * S1 / (sigma2 * S2 * N ** 2))
/ho

Not enough samples for outliercleaned - ACV. Skipping plot generation.
Not enough samples for outliercleaned - CEP. Skipping plot generation.
Not enough samples for outliercleaned - DCEP. Skipping plot generation.
Not enough samples for median_detrended - ACV. Skipping plot generation.
Not enough samples for median_detrended - CEP. Skipping plot generation.
Not enough samples for median_detrended - DCEP. Skipping plot generation.
Not enough samples for raw - ACV. Skipping plot generation.
Not enough samples for raw - CEP. Skipping plot generation.
Not enough samples for raw - DCEP. Skipping plot generation.
Processing completed.


## Summary <a class="anchor" id="fifth-bullet"></a>

At this point, you should have:
* light curve plots that are meaningful: error bars, labels...
* a feature table for the outlier-cleaned light curve
* triangle plots that are meaningful: plot size/ number of types, colors, labels...